# Graph isomorphism PPO


## Setup


In [1]:
FORCE_CPU = True

SEED = 349287

BATCH_SIZE = 32

In [2]:
from typing import Optional

import torch
from torch import Tensor
from torch.utils.data import DataLoader
from torch import nn
from torch.distributions import Categorical

from tensordict.nn import (
    TensorDictModule,
    TensorDictModuleBase,
    TensorDictSequential,
    ProbabilisticTensorDictSequential,
)
from tensordict.nn.distributions import CompositeDistribution
from tensordict.tensordict import TensorDict, TensorDictBase
from tensordict.nn import InteractionType

from torchrl.collectors import SyncDataCollector
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage
from torchrl.data.tensor_specs import (
    CompositeSpec,
    DiscreteTensorSpec,
    BinaryDiscreteTensorSpec,
    MultiDiscreteTensorSpec,
    TensorSpec,
    UnboundedContinuousTensorSpec,
    Box,
)
from torchrl.envs import EnvBase
from torchrl.envs.utils import check_env_specs
from torchrl.modules import ProbabilisticActor, ActorValueOperator
from torchrl.objectives import ValueEstimators

from torch_geometric.loader import DataLoader as GeometricDataLoader
from torch_geometric.data import Batch as GeometricBatch, Data as GeometricData

from jaxtyping import Float, Int, Bool

from tqdm import tqdm

from pvg.graph_isomorphism import (
    GraphIsomorphismAgentBody,
    GraphIsomorphismAgentPolicyHead,
    GraphIsomorphismAgentValueHead,
)
from pvg.parameters import Parameters
from pvg.graph_isomorphism.data import GraphIsomorphismData, GraphIsomorphismDataset
from pvg.utils.data import gi_data_to_tensordict
from pvg.utils.torchrl_objectives import ClipPPOLossMultipleActions
from pvg.constants import VERIFIER_AGENT_NUM, PROVER_AGENT_NUM

/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
torch.manual_seed(SEED)
torch_generator = torch.Generator().manual_seed(SEED)

In [4]:
if not FORCE_CPU and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cpu


In [5]:
params = Parameters(
    scenario="graph_isomorphism",
    trainer="ppo",
    dataset="eru10000",
)

## Specification


In [6]:
class AdjacencyMatrixBox(Box):
    """An abstract representation of the space of adjacency matrices."""

    def __init__(self, max_num_nodes: int):
        self.max_num_nodes = max_num_nodes

In [7]:
class AdjacencyMatrixSpec(TensorSpec):
    def __init__(
        self,
        max_num_nodes: int,
        shape: torch.Size | None = None,
        device: Optional[torch.device | str | int] = None,
        dtype: str | torch.dtype = torch.int32,
    ):
        self.max_num_nodes = max_num_nodes
        self.device = device
        self.dtype = dtype

        if shape is None:
            self.shape = torch.Size([max_num_nodes, max_num_nodes])
        else:
            if shape[-2:] != (max_num_nodes, max_num_nodes):
                raise ValueError(
                    f"The last two dimensions of the shape must be {max_num_nodes}. "
                    f"Got {shape[-2:]}."
                )
            self.shape = torch.Size(shape)

        self.space = AdjacencyMatrixBox(max_num_nodes)

    def is_in(self, val: torch.Tensor) -> bool:
        """Check if a value is a valid adjacency matrix."""

        # Basic type checks
        if not isinstance(val, torch.Tensor):
            return False
        if val.shape[-2:] != (self.max_num_nodes, self.max_num_nodes):
            return False
        if val.dtype != self.dtype:
            return False

        # Make sure the values are either 0 or 1
        if not torch.all(torch.isin(val, torch.tensor([0, 1], device=self.device))):
            return False

        # Make sure the matrix is symmetric
        if not torch.all(val.transpose(-1, -2) == val):
            return False

        # Make sure the diagonal is all zeros
        if not torch.all(torch.isin(torch.diagonal(val, dim1=-2, dim2=-1), 0)):
            return False

        return True

    def rand(self, shape: Optional[list[int] | torch.Size] = None) -> torch.Tensor:
        """Generate a random 1/2 Erdos-Renyi adjacency matrix."""

        if shape is None:
            shape = shape = torch.Size([])

        adjacency_values = torch.rand(*shape, *self.shape, device=device)
        adjacency = (adjacency_values < 0.5).to(self.dtype)
        adjacency = adjacency.triu(diagonal=1)
        adjacency += adjacency.transpose(1, 2).clone()

        return adjacency

    def _project(self, val: torch.Tensor) -> torch.Tensor:
        """Project a value to the space of valid adjacency matrices."""

        # Symmetrize the matrix
        val = (val + val.transpose(1, 2)) / 2

        # Make sure the diagonal is all zeros
        val[..., torch.arange(self.max_num_nodes), torch.arange(self.max_num_nodes)] = 0

        # Make sure the values are either 0 or 1
        return torch.clamp(torch.round(val), min=0, max=1).to(self.dtype)

## Environment


### Variable batch-size data loader

In [8]:
def forgetful_cycle(iterable):
    """A version of cycle that doesn't save copies of the values"""
    while True:
        for i in iterable:
            yield i

In [9]:
class VariableGeometricDataCycler:
    """A loader that cycles through geometric data, but allows the batch size to vary.

    Parameters
    ----------
    dataloader : GeometricDataLoader
        The base dataloader to use. This dataloader will be cycled through.
    """

    def __init__(self, dataloader: GeometricDataLoader):
        self.dataloader = dataloader
        self.dataloader_iter = iter(forgetful_cycle(self.dataloader))
        self.remainder: Optional[list] = None

    def get_batch(self, batch_size: int) -> GeometricBatch:
        """Get a batch of data from the dataloader with the given batch size.

        If the dataloader is exhausted, it will be reset.

        Parameters
        ----------
        batch_size : int
            The size of the batch to return.

        Returns
        -------
        batch : Tensor
            A batch of data with the given batch size.
        """

        left_to_sample = batch_size
        batch_components = []

        # Start by sampling from the remainder from the previous sampling
        if self.remainder is not None:
            batch_components.extend(self.remainder[:left_to_sample])
            if len(self.remainder) <= left_to_sample:
                left_to_sample -= len(self.remainder)
                self.remainder = None
            else:
                self.remainder = self.remainder[left_to_sample:]
                left_to_sample = 0

        # Keep sampling batches until we have enough
        while left_to_sample > 0:
            batch = next(self.dataloader_iter)
            batch_components.extend(batch[:left_to_sample])
            if len(batch) <= left_to_sample:
                left_to_sample -= len(batch)
            else:
                self.remainder = batch[left_to_sample:]
                left_to_sample = 0

        # Concatenate the batch components into a single batch
        batch = GeometricBatch.from_data_list(
            batch_components, follow_batch=self.dataloader.follow_batch
        )
        return batch

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.dataloader!r})"

### Environment

In [10]:
class GraphIsomorphismEnv(EnvBase):
    def __init__(
        self,
        params: Parameters,
        device: torch.device | str = device,
        int_dtype: torch.dtype = torch.int,
    ):
        super().__init__(device=device)
        self.params = params
        self.int_dtype = int_dtype

        # Create a random number generator
        self.rng = torch.Generator(device=device)

        # Load the dataset
        self.dataset = GraphIsomorphismDataset(params)
        self.data_cycler: Optional[VariableGeometricDataCycler] = None

        # Compute the maximum number of nodes in the dataset
        self.max_num_nodes = 0
        for data in self.dataset:
            self.max_num_nodes = max(
                self.max_num_nodes, data.x_a.shape[0], data.x_b.shape[0]
            )

        # The number of environments is the number of episodes we can fit in a batch
        self.num_envs = params.ppo.frames_per_batch // params.max_message_rounds
        self.batch_size = (self.num_envs,)

        # The spec for the observation space: agents see the adjacency matrix and the
        # messages sent so far. The "message" field contains the most recent message.
        self.observation_spec = CompositeSpec(
            adjacency=AdjacencyMatrixSpec(
                self.max_num_nodes,
                shape=(self.num_envs, 2, self.max_num_nodes, self.max_num_nodes),
                dtype=self.int_dtype,
            ),
            x=BinaryDiscreteTensorSpec(
                params.max_message_rounds,
                shape=(
                    self.num_envs,
                    2,
                    self.max_num_nodes,
                    params.max_message_rounds,
                ),
                dtype=torch.float,
            ),
            node_mask=BinaryDiscreteTensorSpec(
                self.max_num_nodes,
                shape=(
                    self.num_envs,
                    2,
                    self.max_num_nodes,
                ),
                dtype=torch.bool,
            ),
            message=DiscreteTensorSpec(
                2 * self.max_num_nodes,
                shape=(self.num_envs,),
                dtype=torch.long,
            ),
            round=DiscreteTensorSpec(
                params.max_message_rounds,
                shape=(self.num_envs,),
                dtype=self.int_dtype,
            ),
            shape=(self.num_envs,),
        ).to(self.device)

        # The spec for the state space: the true label and the round number
        self.state_spec = CompositeSpec(
            y=BinaryDiscreteTensorSpec(
                1,
                shape=(self.num_envs, 1),
                dtype=self.int_dtype,
            ),
            shape=(self.num_envs,),
        ).to(self.device)

        # Each action space has shape (batch_size, num_agents). Each agent chooses both
        # a node and a decision: reject, accept or continue (represented as 0, 1 or 2).
        # The node is is a number between 0 and 2 * max_num_nodes - 1. If it is less
        # than max_num_nodes, it is a node in the first graph, otherwise it is a node in
        # the second graph. The verifier is agent 0 and the prover is agent 1.
        self.action_spec = CompositeSpec(
            agents=CompositeSpec(
                node_selected=DiscreteTensorSpec(
                    2 * self.max_num_nodes,
                    shape=(self.num_envs, 2),
                    dtype=self.int_dtype,
                ),
                decision=DiscreteTensorSpec(
                    3,
                    shape=(self.num_envs, 2),
                    dtype=self.int_dtype,
                ),
                shape=(self.num_envs,),
            ),
            shape=(self.num_envs,),
        ).to(self.device)

        self.reward_spec = CompositeSpec(
            agents=CompositeSpec(
                reward=UnboundedContinuousTensorSpec(shape=(self.num_envs, 2)),
                shape=(self.num_envs,),
            ),
            shape=(self.num_envs,),
        ).to(self.device)

        self.done_spec = CompositeSpec(
            done=BinaryDiscreteTensorSpec(
                self.num_envs, shape=(self.num_envs,), dtype=torch.bool
            ),
            shape=(self.num_envs,),
        ).to(self.device)

    def _step(self, tensordict: TensorDictBase) -> TensorDictBase:
        """Perform a step in the environment."""

        # Extract the tensors from the dict
        y: Int[Tensor, "batch"] = tensordict["y"]
        x: Float[Tensor, "batch graph node message_round"] = tensordict["x"]
        round: Int[Tensor, "batch"] = tensordict["round"]
        node_selected: Int[Tensor, "batch agent"] = tensordict[
            "agents", "node_selected"
        ]
        decision: Int[Tensor, "batch agent"] = tensordict["agents", "decision"]
        done: Bool[Tensor, "batch"] = tensordict["done"]

        # Compute index of the agent whose turn it is. The prover goes first.
        agent_index: Int[Tensor, "batch"] = (round % 2)
        if PROVER_AGENT_NUM == 1:
            agent_index = 1 - agent_index

        # Determine which graph contains the selected node and which node it is there
        # (batch agent)
        which_graph = node_selected >= self.max_num_nodes
        # (batch agent)
        graph_node = torch.where(
            which_graph, node_selected - self.max_num_nodes, node_selected
        )

        # Write the node selected by the agent whose turn it is as a (one-hot) message
        x[
            torch.arange(x.shape[0]),
            which_graph[torch.arange(which_graph.shape[0]), agent_index].int(),
            graph_node[torch.arange(which_graph.shape[0]), agent_index],
            round,
        ] = 1

        # Set the node selected by the agent whose turn it is as the message
        message = node_selected[
            torch.arange(node_selected.shape[0]), agent_index
        ].long()

        # If the verifier has made a guess, compute the reward and terminate the episode
        verifier_decision_made = (agent_index == VERIFIER_AGENT_NUM) & (
            decision[:, VERIFIER_AGENT_NUM] != 2
        )
        done = done | verifier_decision_made
        reward_verifier = (
            verifier_decision_made & (decision[:, VERIFIER_AGENT_NUM] == y.squeeze())
        ).float()
        reward_verifier = reward_verifier * self.params.verifier_reward
        reward_prover = (
            verifier_decision_made & (decision[:, VERIFIER_AGENT_NUM] == 1)
        ).float()
        reward_prover = reward_prover * self.params.prover_reward

        # If we reach the end of the episode and the verifier has not made a guess,
        # terminate it with a negative reward for the verifier
        done = done | (round >= self.params.max_message_rounds - 1)
        reward_verifier[
            (round >= self.params.max_message_rounds - 1) & ~verifier_decision_made
        ] = self.params.verifier_terminated_penalty

        # Stack the rewards for the two agents
        if PROVER_AGENT_NUM == 1:
            reward = torch.stack([reward_verifier, reward_prover], dim=-1)
        else:
            reward = torch.stack([reward_prover, reward_verifier], dim=-1)

        # Put everything together
        next = TensorDict(
            dict(
                adjacency=tensordict["adjacency"],
                x=x,
                node_mask=tensordict["node_mask"],
                message=message,
                round=round + 1,
                done=done,
                agents=TensorDict(dict(reward=reward), batch_size=self.batch_size),
            ),
            batch_size=self.batch_size,
        )
        return next

    def _reset(self, tensordict: Optional[TensorDictBase] = None) -> TensorDictBase:
        """(Partially) reset the environment.

        For each episode which is done, takes a new sample from the dataset and resets
        the episode.
        """

        # If no tensordict is given, we're starting afresh
        if tensordict is None:
            tensordict = TensorDict(
                dict(
                    adjacency=torch.empty(
                        *self.batch_size,
                        2,
                        self.max_num_nodes,
                        self.max_num_nodes,
                        device=self.device,
                        dtype=self.int_dtype,
                    ),
                    x=torch.empty(
                        *self.batch_size,
                        2,
                        self.max_num_nodes,
                        self.params.max_message_rounds,
                        device=self.device,
                        dtype=torch.float,
                    ),
                    node_mask=torch.empty(
                        *self.batch_size,
                        2,
                        self.max_num_nodes,
                        device=self.device,
                        dtype=torch.bool,
                    ),
                    message=torch.empty(
                        *self.batch_size,
                        device=self.device,
                        dtype=torch.long,
                    ),
                    y=torch.empty(
                        *self.batch_size,
                        1,
                        device=self.device,
                        dtype=self.int_dtype,
                    ),
                    round=torch.empty(
                        *self.batch_size,
                        device=self.device,
                        dtype=self.int_dtype,
                    ),
                    done=torch.empty(
                        *self.batch_size,
                        device=self.device,
                        dtype=torch.bool,
                    ),
                ),
                batch_size=self.batch_size,
            )

            new_mask = torch.ones(
                *self.batch_size, dtype=torch.bool, device=self.device
            )

        else:
            new_mask = tensordict["done"]
            tensordict = tensordict.clone()

        # If we don't have a data cycler yet, create one
        if self.data_cycler is None:
            dataloader = GeometricDataLoader(
                self.dataset,
                batch_size=self.num_envs,
                follow_batch=["x_a", "x_b"],
                shuffle=True,
                generator=self.rng,
            )
            self.data_cycler = VariableGeometricDataCycler(dataloader)

        # Sample a new batch of data for the episodes that are done
        batch = self.data_cycler.get_batch(new_mask.sum().item())
        batch_tensordict = gi_data_to_tensordict(
            batch, node_dim_size=self.max_num_nodes
        )

        # Copy the new data into the output
        tensordict["adjacency"][new_mask] = batch_tensordict["adjacency"]
        tensordict["x"][new_mask] = torch.zeros_like(tensordict["x"][new_mask])
        tensordict["node_mask"][new_mask] = batch_tensordict["node_mask"]
        tensordict["message"][new_mask] = 0
        tensordict["y"][new_mask] = (batch.wl_score == -1).int().unsqueeze(-1)
        tensordict["round"][new_mask] = 0
        tensordict["done"][new_mask] = False

        return tensordict

    def _set_seed(self, seed: int | None):
        self.rng.manual_seed(seed)

### Testing


In [11]:
env = GraphIsomorphismEnv(params)

In [12]:
check_env_specs(env)

check_env_specs succeeded!


In [13]:
def printer(env, tensordict):
    to_print = []
    for i in range(2):
        to_print.append(
            f"[{tensordict['round'][i].item()}] "
            f"y: {tensordict['y'][i].item()} "
            f"P: ({tensordict['agents', 'node_selected'][i, PROVER_AGENT_NUM].item():>2}) "
            f" {tensordict['next', 'agents', 'reward'][i, PROVER_AGENT_NUM].item():>2} "
            f"V: ({tensordict['agents', 'node_selected'][i, VERIFIER_AGENT_NUM].item():>2}, "
            f" {tensordict['agents', 'decision'][i, VERIFIER_AGENT_NUM].item():>2}) "
            f" {tensordict['next', 'agents', 'reward'][i, VERIFIER_AGENT_NUM].item():>2} "
        )
    print(" | ".join(to_print))
    # print(tensordict["message"][:2, ..., :3].transpose(-1, -2))
    # print(tensordict["message"][:2, ..., :3])


with torch.no_grad():
    out = env.rollout(
        max_steps=40,
        callback=printer,
        auto_cast_to_device=True,
        break_when_any_done=False,
    )

[0] y: 0 P: (11)  0.0 V: ( 5,   1)  0.0  | [0] y: 1 P: (12)  0.0 V: (12,   0)  0.0 
[1] y: 0 P: ( 9)  0.0 V: ( 1,   0)  1.0  | [1] y: 1 P: (14)  1.0 V: (18,   1)  1.0 
[0] y: 0 P: (17)  0.0 V: ( 5,   1)  0.0  | [0] y: 1 P: ( 0)  0.0 V: (13,   2)  0.0 
[1] y: 0 P: (21)  1.0 V: (11,   1)  0.0  | [1] y: 1 P: (19)  0.0 V: (16,   2)  0.0 
[0] y: 0 P: ( 3)  0.0 V: (14,   0)  0.0  | [2] y: 1 P: (13)  0.0 V: (10,   0)  0.0 
[1] y: 0 P: ( 3)  1.0 V: (11,   1)  0.0  | [3] y: 1 P: (17)  0.0 V: ( 7,   0)  0.0 
[0] y: 0 P: ( 6)  0.0 V: (13,   2)  0.0  | [0] y: 1 P: (16)  0.0 V: (16,   2)  0.0 
[1] y: 0 P: (17)  0.0 V: ( 0,   0)  1.0  | [1] y: 1 P: ( 9)  0.0 V: (15,   0)  0.0 
[0] y: 0 P: (17)  0.0 V: (21,   1)  0.0  | [0] y: 1 P: (16)  0.0 V: (10,   2)  0.0 
[1] y: 0 P: (17)  0.0 V: ( 2,   0)  1.0  | [1] y: 1 P: ( 0)  0.0 V: ( 3,   0)  0.0 
[0] y: 0 P: (17)  0.0 V: (20,   1)  0.0  | [0] y: 1 P: ( 1)  0.0 V: ( 9,   1)  0.0 
[1] y: 0 P: (12)  0.0 V: ( 0,   0)  1.0  | [1] y: 1 P: ( 6)  0.0 V: (16,   0

In [14]:
out

TensorDict(
    fields={
        adjacency: Tensor(shape=torch.Size([125, 40, 2, 11, 11]), device=cpu, dtype=torch.int32, is_shared=False),
        agents: TensorDict(
            fields={
                decision: Tensor(shape=torch.Size([125, 40, 2]), device=cpu, dtype=torch.int32, is_shared=False),
                node_selected: Tensor(shape=torch.Size([125, 40, 2]), device=cpu, dtype=torch.int32, is_shared=False)},
            batch_size=torch.Size([125, 40]),
            device=cpu,
            is_shared=False),
        done: Tensor(shape=torch.Size([125, 40]), device=cpu, dtype=torch.bool, is_shared=False),
        message: Tensor(shape=torch.Size([125, 40]), device=cpu, dtype=torch.int64, is_shared=False),
        next: TensorDict(
            fields={
                adjacency: Tensor(shape=torch.Size([125, 40, 2, 11, 11]), device=cpu, dtype=torch.int32, is_shared=False),
                agents: TensorDict(
                    fields={
                        reward: Tensor(sha

## Policy and critic


### Body

In [15]:
class GraphIsomorphismCombinedBody(TensorDictModuleBase):
    in_keys = ("round", "x", "adjacency", "message", "node_mask")
    out_keys = ("round", ("agents", "node_level_repr"), ("agents", "graph_level_repr"))

    def __init__(
        self,
        prover_body: GraphIsomorphismAgentBody,
        verifier_body: GraphIsomorphismAgentBody,
    ) -> None:
        super().__init__()
        self.prover_body = prover_body
        self.verifier_body = verifier_body

    def forward(self, data: TensorDictBase) -> TensorDict:
        round: Int[Tensor, "batch"] = data["round"]

        # Build tensordicts to feed to each agent when it's their turn
        input_prover_dict = {}
        input_verifier_dict = {}
        for key in self.prover_body.in_keys:
            if key == "ignore_message":
                input_prover_dict[key] = torch.zeros_like(round, dtype=torch.bool)
                input_verifier_dict[key] = round == 0
            else:
                input_prover_dict[key] = data[key]
                input_verifier_dict[key] = data[key]
        input_prover = TensorDict(
            input_prover_dict,
            batch_size=data.batch_size,
        )
        input_verifier = TensorDict(
            input_verifier_dict,
            batch_size=data.batch_size,
        )

        # Run the prover and verifier bodies
        output_prover = self.prover_body(input_prover)
        output_verifier = self.verifier_body(input_verifier)

        # Stack the outputs
        node_level_repr = torch.stack(
            [output_prover["node_level_repr"], output_verifier["node_level_repr"]],
            dim=-3,
        )
        graph_level_repr = torch.stack(
            [output_prover["graph_level_repr"], output_verifier["graph_level_repr"]],
            dim=-2,
        )
        if PROVER_AGENT_NUM == 1:
            node_level_repr = node_level_repr.flip(-3)
            graph_level_repr = graph_level_repr.flip(-2)

        return data.update(
            dict(
                agents=dict(
                    node_level_repr=node_level_repr,
                    graph_level_repr=graph_level_repr,
                )
            )
        )

### Policy head

In [16]:
class GraphIsomorphismCombinedPolicyHead(TensorDictModuleBase):
    in_keys = (("agents", "node_level_repr"), ("agents", "graph_level_repr"))
    out_keys = (("agents", "node_selected_logits"), ("agents", "decision_logits"))

    def __init__(
        self,
        prover_policy_head: GraphIsomorphismAgentPolicyHead,
        verifier_policy_head: GraphIsomorphismAgentPolicyHead,
    ):
        super().__init__()
        self.prover_policy_head = prover_policy_head
        self.verifier_policy_head = verifier_policy_head

    def forward(self, head_output: TensorDictBase) -> TensorDict:
        """Run the prover and verifier policy heads and combine their outputs.

        Parameters
        ----------
        tensordict : TensorDictBase
            The input to the value heads. Should contain the keys:

            - ("agents", "node_level_repr"): The node-level representation from the
              body.
            - ("agents", "graph_level_repr"): The node-level representation from the
              body.

        Returns
        -------
        tensordict: TensorDict
            The tensordict update in place with the output of the value heads.
        """

        # Run the policy heads to obtain the probability distributions
        input_prover = TensorDict(
            dict(
                node_level_repr=head_output["agents", "node_level_repr"][
                    ..., PROVER_AGENT_NUM, :, :, :
                ],
                graph_level_repr=head_output["agents", "graph_level_repr"][
                    ..., PROVER_AGENT_NUM, :, :
                ],
            ),
            batch_size=head_output.batch_size,
        )
        input_verifier = TensorDict(
            dict(
                node_level_repr=head_output["agents", "node_level_repr"][
                    ..., VERIFIER_AGENT_NUM, :, :, :
                ],
                graph_level_repr=head_output["agents", "graph_level_repr"][
                    ..., VERIFIER_AGENT_NUM, :, :
                ],
            ),
            batch_size=head_output.batch_size,
        )
        output_prover = self.prover_policy_head(input_prover)
        output_verifier = self.verifier_policy_head(input_verifier)

        # Stack the outputs
        node_selected_logits = torch.stack(
            [
                output_prover["node_selected_logits"],
                output_verifier["node_selected_logits"],
            ],
            dim=-2,
        )
        decision_logits = torch.stack(
            [
                torch.zeros_like(output_verifier["decision_logits"]),
                output_verifier["decision_logits"],
            ],
            dim=-2,
        )
        if PROVER_AGENT_NUM == 1:
            node_selected_logits = node_selected_logits.flip(-2)
            decision_logits = decision_logits.flip(-2)

        return head_output.update(
            dict(
                agents=TensorDict(
                    dict(
                        node_selected_logits=node_selected_logits,
                        decision_logits=decision_logits,
                    ),
                    batch_size=head_output.batch_size,
                )
            )
        )

### Value head

In [17]:
class GraphIsomorphismCombinedValueHead(TensorDictModuleBase):
    in_keys = (("agents", "graph_level_repr"),)
    out_keys = (("agents", "value"),)

    def __init__(
        self,
        prover_value_head: GraphIsomorphismAgentValueHead,
        verifier_value_head: GraphIsomorphismAgentValueHead,
    ):
        super().__init__()
        self.prover_value_head = prover_value_head
        self.verifier_value_head = verifier_value_head

    def forward(self, head_output: TensorDictBase) -> TensorDict:
        """Run the prover and verifier value heads and combine their values.

        Parameters
        ----------
        tensordict : TensorDictBase
            The input to the value heads. Should contain the keys:

            - ("agents", "graph_level_repr"): The node-level representation from the
              body.

        Returns
        -------
        tensordict: TensorDict
            The tensordict update in place with the output of the value heads.
        """

        # Run the policy heads to obtain the value estimates
        input_prover = TensorDict(
            dict(
                graph_level_repr=head_output["agents", "graph_level_repr"][
                    ..., PROVER_AGENT_NUM, :, :
                ],
            ),
            batch_size=head_output.batch_size,
        )
        input_verifier = TensorDict(
            dict(
                graph_level_repr=head_output["agents", "graph_level_repr"][
                    ..., VERIFIER_AGENT_NUM, :, :
                ],
            ),
            batch_size=head_output.batch_size,
        )
        output_prover = self.prover_value_head(input_prover)
        output_verifier = self.verifier_value_head(input_verifier)

        # Stack the outputs
        value = torch.stack(
            [output_prover["value"], output_verifier["value"]],
            dim=-1,
        )
        if PROVER_AGENT_NUM == 1:
            value = value.flip(-2)

        return head_output.update(
            dict(
                agents=TensorDict(
                    dict(value=value),
                    batch_size=head_output.batch_size,
                )
            ),
        )

### Composite categorical distribution

In [18]:
class CompositeCategoricalDistribution(CompositeDistribution):
    """A composition of categorical distributions.

    Allows specifying the parameters of the categorical distributions either as logits
    or as probabilities.

    The `log_prob` method is reimplemented with the following changes: 
    
    - The log-probability can be stored in a different key (specified by the
      "log_prob_key" parameter)
    - It only computes stores the total log-probability, not the individual ones
    - It doesn't reduce all non-batch dimensions in the log-probability.

    Parameter names must be strings ending in "_logits" or "_probs". However, the
    suffix-stripped can be changed by passing a key transform function or a lookup
    table. For example, to specify the parameters of a categorical distribution over key
    `("agents", "action")` using logits, you can pass the following:
    
    >>> CompositeCategoricalDistribution(
    ...     action_logits=..., 
    ...     key_transform=lambda x: ("agents", x)
    ... )

    Parameters
    ----------
    **categorical_params : dict[str, Tensor]
        The parameters of the categorical distributions. Each key is the name of a
        categorical parameter appended with "_logits" or "_probs" and each value is a
        Tensor containing the logits or probabilities of the categorical distribution.
    key_transform : callable[[str], [str | tuple[str]]] | dict[str, str | tuple[str]],
    optional
        A function that transforms the keys of the categorical parameters. If a dict is
        given, it is used as a lookup table. If a callable is given, it is applied to
        each key. Defaults to the identity function.
    log_prob_key: NestedKey, default="sample_log_prob"
        The tensordict key to use for the log-probability of the sample.
    """

    def __init__(self, **kwargs):
        # Get the key transform
        try:
            key_transform = kwargs.pop("key_transform")
            if isinstance(key_transform, dict):
                key_transform = lambda x: key_transform[x]
            elif not callable(key_transform):
                raise ValueError("key_transform must be a callable or a dict.")
        except KeyError:
            key_transform = lambda x: x

        # Get the log-probability key
        self.log_prob_key = kwargs.pop("log_prob_key", "sample_log_prob")

        batch_size = None
        composite_params = {}
        name_suffixes = ("logits", "probs")
        for name, param_value in kwargs.items():
            for name_suffix in name_suffixes:
                if name.endswith("_" + name_suffix):
                    # Set the parameters of the categorical distribution
                    resolved_param_name = key_transform(name[: -len(name_suffix) - 1])
                    composite_params[resolved_param_name] = {name_suffix: param_value}

                    # Make sure all the categorical parameters have the same batch size
                    if batch_size is None:
                        batch_size = param_value.shape[0]
                    elif batch_size != param_value.shape[0]:
                        raise ValueError(
                            "All categorical parameters must have the same batch size."
                        )

        composite_params_td = TensorDict(composite_params, batch_size=batch_size)

        super().__init__(
            params=composite_params_td,
            distribution_map={key: Categorical for key in composite_params},
        )


    def log_prob(self, sample: TensorDictBase) -> TensorDictBase:
        """Computes the log probability of a sample for the composite distribution
        
        Parameters
        ----------
        sample: TensorDictBase
            A tensordict containing the sample
        
        Returns
        -------
        updated_sample: TensorDictBase
            The sample tensordict updated with the log probability of the sample
        """
        slp = 0.0
        d = {}
        for name, dist in self.dists.items():
            slp += dist.log_prob(sample.get(name))
        d[self.log_prob_key] = slp
        sample.update(d)
        return sample

In [19]:
d = CompositeCategoricalDistribution(
    node_selected_logits=torch.randn([125, 2, 22]),
    decision_logits=torch.randn([125, 2, 3]),
)
d.sample()

TensorDict(
    fields={
        decision: Tensor(shape=torch.Size([125, 2]), device=cpu, dtype=torch.int64, is_shared=False),
        node_selected: Tensor(shape=torch.Size([125, 2]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([125]),
    device=None,
    is_shared=False)

### Instantiating actors and value estimators

In [20]:
class DefaultInteractionMixin(ProbabilisticTensorDictSequential):
    """A mixin for accessing the last module's default interaction type.

    Allows access to 'default_interaction_mode' and 'default_interaction_type'
    attributes for the last module in the sequential.
    """

    @property
    def default_interaction_mode(self) -> str:
        """The default interaction mode of the last module in the sequential."""
        return self[-1].default_interaction_mode

    @property
    def default_interaction_type(self) -> InteractionType:
        """The default interaction type of the last module in the sequential."""
        return self[-1].default_interaction_type

In [21]:
class ProbabilisticActorDefaultInteractionType(
    ProbabilisticActor, DefaultInteractionMixin
):
    pass

In [22]:
prover_body = GraphIsomorphismAgentBody(params, "prover", device=device)
verifier_body = GraphIsomorphismAgentBody(params, "verifier", device=device)
prover_policy_head = GraphIsomorphismAgentPolicyHead(params, "prover", device=device)
verifier_policy_head = GraphIsomorphismAgentPolicyHead(
    params, "verifier", device=device
)
prover_value_head = GraphIsomorphismAgentValueHead(params, "prover", device=device)
verifier_value_head = GraphIsomorphismAgentValueHead(
    params, "verifier", device=device
)

In [23]:
body = GraphIsomorphismCombinedBody(prover_body, verifier_body)

In [24]:
policy_head = ProbabilisticActorDefaultInteractionType(
    GraphIsomorphismCombinedPolicyHead(prover_policy_head, verifier_policy_head),
    spec=env.action_spec,
    distribution_class=CompositeCategoricalDistribution,
    distribution_kwargs=dict(
        key_transform=lambda x: ("agents", x),
        log_prob_key=("agents", "sample_log_prob"),
    ),
    in_keys=dict(
        node_selected_logits=("agents", "node_selected_logits"),
        decision_logits=("agents", "decision_logits"),
    ),
    out_keys=env.action_keys,
    return_log_prob=True,
    log_prob_key=("agents", "sample_log_prob"),
)

In [25]:
# policy_prob_module = SafeProbabilisticModule(
#     spec=env.action_spec,
#     distribution_class=CompositeCategoricalDistribution,
#     in_keys=dict(
#         node_selected_logits=("agents", "node_selected_logits"),
#         decision_logits=("agents", "decision_logits"),
#     ),
#     out_keys=env.action_keys,
#     return_log_prob=True,
#     log_prob_key=("agents", "sample_log_prob"),
# )

In [26]:
value_head = GraphIsomorphismCombinedValueHead(
    prover_value_head, verifier_value_head
)

In [27]:
full_model = ActorValueOperator(body, policy_head, value_head)

In [28]:
policy = full_model.get_policy_operator()

### Testing

In [29]:
policy(env.reset())

TensorDict(
    fields={
        adjacency: Tensor(shape=torch.Size([125, 2, 11, 11]), device=cpu, dtype=torch.int32, is_shared=False),
        agents: TensorDict(
            fields={
                decision: Tensor(shape=torch.Size([125, 2]), device=cpu, dtype=torch.int64, is_shared=False),
                decision_logits: Tensor(shape=torch.Size([125, 2, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                graph_level_repr: Tensor(shape=torch.Size([125, 2, 2, 16]), device=cpu, dtype=torch.float32, is_shared=False),
                node_level_repr: Tensor(shape=torch.Size([125, 2, 2, 11, 16]), device=cpu, dtype=torch.float32, is_shared=False),
                node_selected: Tensor(shape=torch.Size([125, 2]), device=cpu, dtype=torch.int64, is_shared=False),
                node_selected_logits: Tensor(shape=torch.Size([125, 2, 22]), device=cpu, dtype=torch.float32, is_shared=False),
                sample_log_prob: Tensor(shape=torch.Size([125, 2]), device=cpu, dty

In [30]:
full_model.get_value_operator()(env.reset())

TensorDict(
    fields={
        adjacency: Tensor(shape=torch.Size([125, 2, 11, 11]), device=cpu, dtype=torch.int32, is_shared=False),
        agents: TensorDict(
            fields={
                graph_level_repr: Tensor(shape=torch.Size([125, 2, 2, 16]), device=cpu, dtype=torch.float32, is_shared=False),
                node_level_repr: Tensor(shape=torch.Size([125, 2, 2, 11, 16]), device=cpu, dtype=torch.float32, is_shared=False),
                value: Tensor(shape=torch.Size([125, 2]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([125]),
            device=None,
            is_shared=False),
        done: Tensor(shape=torch.Size([125]), device=cpu, dtype=torch.bool, is_shared=False),
        message: Tensor(shape=torch.Size([125]), device=cpu, dtype=torch.int64, is_shared=False),
        node_mask: Tensor(shape=torch.Size([125, 2, 11]), device=cpu, dtype=torch.bool, is_shared=False),
        round: Tensor(shape=torch.Size([125]), device

In [31]:
# policy = ProbabilisticTensorDictSequential(full_model, policy_head, policy_prob_module)

## Replay buffer

In [32]:
replay_buffer = ReplayBuffer(
    storage=LazyTensorStorage(
        params.ppo.frames_per_batch, device=device
    ),
    sampler=SamplerWithoutReplacement(),
    batch_size=params.ppo.minibatch_size,
)
replay_buffer

ReplayBuffer(storage=<torchrl.data.replay_buffers.storages.LazyTensorStorage object at 0x7f6f500f1410>, sampler=<torchrl.data.replay_buffers.samplers.SamplerWithoutReplacement object at 0x7f6f5b2e97d0>, writer=<torchrl.data.replay_buffers.writers.RoundRobinWriter object at 0x7f6f500f0450>)

## Data collector

In [33]:
collector = SyncDataCollector(
    env,
    policy,
    device=device,
    storing_device=device,
    frames_per_batch=params.ppo.frames_per_batch,
    total_frames=params.ppo.frames_per_batch * params.ppo.num_iterations,
)

## Loss function

In [34]:
loss_module = ClipPPOLossMultipleActions(
    actor=full_model.get_policy_operator(),
    critic=full_model.get_value_operator(),
    clip_epsilon=params.ppo.clip_epsilon,
    entropy_coef=params.ppo.entropy_eps,
    normalize_advantage=False,
)
loss_module.set_keys(
    reward=env.reward_key,
    action=env.action_keys,
    sample_log_prob=("agents", "sample_log_prob"),
    value=("agents", "value"),
    done=("agents", "done"),
    terminated=("agents", "terminated"),
)


loss_module.make_value_estimator(
    ValueEstimators.GAE, gamma=params.ppo.gamma, lmbda=params.ppo.lmbda
)
gae = loss_module.value_estimator

optim = torch.optim.Adam(loss_module.parameters(), params.ppo.lr)

In [35]:
value_head.prover_value_head.value_mlp.module[0].__dict__["_is_stateless"]

False

In [36]:
value_head.prover_value_head.value_mlp.module[0]

Rearrange('... pair d_in -> ... (pair d_in)')

In [37]:
value_head.prover_value_head.value_mlp.module[0].__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_pre_hooks', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_hooks_always_called', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_state_dict_hooks', '_state_dict_pre_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'pattern', 'axes_lengths', '_multirecipe', '_axes_lengths', '_functionalized', '_decorated_funs', 'forward', '_is_stateless'])

In [38]:
value_head.prover_value_head.value_mlp.module[0].__dict__["_test"] = True

## Training loop

In [39]:
pbar = tqdm(total=params.ppo.num_iterations, desc="episode_reward_mean = 0")

episode_reward_mean_list = []
for tensordict_data in collector:
    tensordict_data.set(
        ("next", "agents", "done"),
        tensordict_data.get(("next", "done"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),
    )
    tensordict_data.set(
        ("next", "agents", "terminated"),
        tensordict_data.get(("next", "terminated"))
        .unsqueeze(-1)
        .expand(tensordict_data.get_item_shape(("next", env.reward_key))),
    )
    # We need to expand the done and terminated to match the reward shape (this is expected by the value estimator)

    with torch.no_grad():
        gae(
            tensordict_data,
            params=loss_module.critic_params,
            target_params=loss_module.target_critic_params,
        )  # Compute GAE and add it to the data

    data_view = tensordict_data.reshape(-1)  # Flatten the batch size to shuffle data
    replay_buffer.extend(data_view)

    for _ in range(params.ppo.num_epochs):
        for _ in range(params.ppo.frames_per_batch // params.ppo.minibatch_size):
            subdata = replay_buffer.sample()
            loss_vals = loss_module(subdata)

            loss_value = (
                loss_vals["loss_objective"]
                + loss_vals["loss_critic"]
                + loss_vals["loss_entropy"]
            )

            loss_value.backward()

            torch.nn.utils.clip_grad_norm_(
                loss_module.parameters(), params.ppo.max_grad_norm
            )  # Optional

            optim.step()
            optim.zero_grad()

    collector.update_policy_weights_()

    # Logging
    done = tensordict_data.get(("next", "agents", "done"))
    episode_reward_mean = (
        tensordict_data.get(("next", "agents", "episode_reward"))[done].mean().item()
    )
    episode_reward_mean_list.append(episode_reward_mean)
    pbar.set_description(f"episode_reward_mean = {episode_reward_mean}", refresh=False)
    pbar.update()

episode_reward_mean = 0:   0%|          | 0/8 [00:00<?, ?it/s]

/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/torch/nn/functional.py:5504: UserWarning: There is a performance drop because we have not yet implemented the batching rule for aten::_scaled_dot_product_attention_math. Please file us an issue on GitHub so that we can prioritize its implementation. (Triggered internally at ../aten/src/ATen/functorch/BatchedFallback.cpp:82.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


KeyError: '2'